In [34]:
import sqlite3
import json
from collections import defaultdict

In [35]:
path_S0 = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'

In [36]:
conn_S0 = sqlite3.connect(path_S0)
c_S0 = conn_S0.cursor()

In [37]:
with open("pi_gen_names.json") as fp:
    pi_gen_names = json.load(fp)
pi_gen_names = {(int((s := k.split(','))[0]), int(s[1])): v for k, v in pi_gen_names.items()}

In [38]:
sql = f"select id, s, t from S0_pi_generators"
name_i = defaultdict(int)
id_to_name = {}
for id, s, t in c_S0.execute(sql):
    if (t - s, s) in pi_gen_names:
        id_to_name[id] = pi_gen_names[(t - s, s)]
    else:
        if (name_i[t-s] > 0):
            id_to_name[id] = f"\\rho_{{{t-s}, {name_i[t-s]}}}"
        else:
            id_to_name[id] = f"\\rho_{{{t-s}}}"
    name_i[t-s] += 1

In [39]:
sql = f"Update S0_pi_generators SET name=?2 where id=?1"
c_S0.executemany(sql, id_to_name.items())

In [40]:
c_S0.close()
conn_S0.commit()